In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split

from sklearn.neural_network import MLPClassifier
import tensorflow as tf

In [2]:
data = pd.read_csv("FIFA.csv")

In [3]:
data

,Date,Team,Opponent,Goal Scored,Ball Possession %,Attempts,On-Target,Off-Target,Blocked,Corners,...,Yellow Card,Yellow & Red,Red,Man of the Match,1st Goal,Round,PSO,Goals in PSO,Own goals,Own goal Time
0,14-06-2018,Russia,Saudi Arabia,5,40,13,7,3,3,6,...,0,0,0,Yes,12.0,Group Stage,No,0,NaN,NaN
1,14-06-2018,Saudi Arabia,Russia,0,60,6,0,3,3,2,...,0,0,0,No,NaN,Group Stage,No,0,NaN,NaN
2,15-06-2018,Egypt,Uruguay,0,43,8,3,3,2,0,...,2,0,0,No,NaN,Group Stage,No,0,NaN,NaN
3,15-06-2018,Uruguay,Egypt,1,57,14,4,6,4,5,...,0,0,0,Yes,89.0,Group Stage,No,0,NaN,NaN
4,15-06-2018,Morocco,Iran,0,64,13,3,6,4,5,...,1,0,0,No,NaN,Group Stage,No,0,1.0,90.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,11-07-2018,England,Croatia,1,46,11,1,6,4,4,...,1,0,0,No,5.0,Semi- Finals,No,0,NaN,NaN
124,14-07-2018,Belgium,England,2,43,12,4,3,5,4,...,1,0,0,Yes,4.0,3rd Place,No,0,NaN,NaN
125,14-07-2018,England,Belgium,0,57,15,5,7,3,5,...,2,0,0,No,NaN,3rd Place,No,0,NaN,NaN
126,15-07-2018,France,Croatia,4,39,8,6,1,1,2,...,2,0,0,Yes,18.0,Final,No,0,1.0,18.0


In [4]:
data.drop('Date', axis=1, inplace=True)

In [5]:
data.isnull().sum()

Team                        0
Opponent                    0
Goal Scored                 0
Ball Possession %           0
Attempts                    0
On-Target                   0
Off-Target                  0
Blocked                     0
Corners                     0
Offsides                    0
Free Kicks                  0
Saves                       0
Pass Accuracy %             0
Passes                      0
Distance Covered (Kms)      0
Fouls Committed             0
Yellow Card                 0
Yellow & Red                0
Red                         0
Man of the Match            0
1st Goal                   34
Round                       0
PSO                         0
Goals in PSO                0
Own goals                 116
Own goal Time             116
dtype: int64

In [6]:
data.drop(['Own goals', 'Own goal Time'], axis=1, inplace=True)

In [7]:
data['1st Goal'] = data['1st Goal'].fillna(data['1st Goal'].mean())

In [8]:
data.dtypes

Team                       object
Opponent                   object
Goal Scored                 int64
Ball Possession %           int64
Attempts                    int64
On-Target                   int64
Off-Target                  int64
Blocked                     int64
Corners                     int64
Offsides                    int64
Free Kicks                  int64
Saves                       int64
Pass Accuracy %             int64
Passes                      int64
Distance Covered (Kms)      int64
Fouls Committed             int64
Yellow Card                 int64
Yellow & Red                int64
Red                         int64
Man of the Match           object
1st Goal                  float64
Round                      object
PSO                        object
Goals in PSO                int64
dtype: object

In [9]:
print(f"Team: {data['Team'].unique()}\n")
print(f"Opponent: {data['Opponent'].unique()}\n")
print(f"Man of the Match: {data['Man of the Match'].unique()}\n")
print(f"Round: {data['Round'].unique()}\n")
print(f"PSO: {data['PSO'].unique()}\n")

Team: ['Russia' 'Saudi Arabia' 'Egypt' 'Uruguay' 'Morocco' 'Iran' 'Portugal'
 'Spain' 'France' 'Australia' 'Argentina' 'Iceland' 'Peru' 'Denmark'
 'Croatia' 'Nigeria' 'Costa Rica' 'Serbia' 'Germany' 'Mexico' 'Brazil'
 'Switzerland' 'Sweden' 'Korea Republic' 'Belgium' 'Panama' 'Tunisia'
 'England' 'Colombia' 'Japan' 'Poland' 'Senegal']

Opponent: ['Saudi Arabia' 'Russia' 'Uruguay' 'Egypt' 'Iran' 'Morocco' 'Spain'
 'Portugal' 'Australia' 'France' 'Iceland' 'Argentina' 'Denmark' 'Peru'
 'Nigeria' 'Croatia' 'Serbia' 'Costa Rica' 'Mexico' 'Germany'
 'Switzerland' 'Brazil' 'Korea Republic' 'Sweden' 'Panama' 'Belgium'
 'England' 'Tunisia' 'Japan' 'Colombia' 'Senegal' 'Poland']

Man of the Match: ['Yes' 'No']

Round: ['Group Stage' 'Round of 16' 'Quarter Finals' 'Semi- Finals' '3rd Place'
 'Final']

PSO: ['No' 'Yes']



In [10]:
label_encoder = LabelEncoder()

data['Man of the Match'] = label_encoder.fit_transform(data['Man of the Match'])
man_mappings = {index: label for index, label in enumerate(label_encoder.classes_)}

data['PSO'] = label_encoder.fit_transform(data['PSO'])
pso_mappings = {index: label for index, label in enumerate(label_encoder.classes_)}

In [11]:
print(f"Man of the Match: {data['Man of the Match'].unique()}\n")
print(f"PSO: {data['PSO'].unique()}\n")

Man of the Match: [1 0]

PSO: [0 1]



In [12]:
round_values = list(data['Round'].unique())
round_values

['Group Stage',
 'Round of 16',
 'Quarter Finals',
 'Semi- Finals',
 '3rd Place',
 'Final']

In [13]:
round_mappings = {label: index for index, label in enumerate(round_values)}
round_mappings

{'Group Stage': 0,
 'Round of 16': 1,
 'Quarter Finals': 2,
 'Semi- Finals': 3,
 '3rd Place': 4,
 'Final': 5}

In [14]:
data['Round'] = data['Round'].apply(lambda x: round_mappings[x])

In [16]:
data['Team'].unique()

array(['Russia', 'Saudi Arabia', 'Egypt', 'Uruguay', 'Morocco', 'Iran',
       'Portugal', 'Spain', 'France', 'Australia', 'Argentina', 'Iceland',
       'Peru', 'Denmark', 'Croatia', 'Nigeria', 'Costa Rica', 'Serbia',
       'Germany', 'Mexico', 'Brazil', 'Switzerland', 'Sweden',
       'Korea Republic', 'Belgium', 'Panama', 'Tunisia', 'England',
       'Colombia', 'Japan', 'Poland', 'Senegal'], dtype=object)

In [17]:
pd.get_dummies(data['Team'])

,Argentina,Australia,Belgium,Brazil,Colombia,Costa Rica,Croatia,Denmark,Egypt,England,...,Portugal,Russia,Saudi Arabia,Senegal,Serbia,Spain,Sweden,Switzerland,Tunisia,Uruguay
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
124,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
125,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
126,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
data['Opponent'].apply(lambda x: "opp_" + x) ## sobar aage ekta prefix opp add korlam

0      opp_opp_Saudi Arabia
1            opp_opp_Russia
2           opp_opp_Uruguay
3             opp_opp_Egypt
4              opp_opp_Iran
               ...         
123         opp_opp_Croatia
124         opp_opp_England
125         opp_opp_Belgium
126         opp_opp_Croatia
127          opp_opp_France
Name: Opponent, Length: 128, dtype: object

In [21]:
pd.get_dummies(data['Opponent'].apply(lambda x: "opp_" + x))

,opp_opp_Argentina,opp_opp_Australia,opp_opp_Belgium,opp_opp_Brazil,opp_opp_Colombia,opp_opp_Costa Rica,opp_opp_Croatia,opp_opp_Denmark,opp_opp_Egypt,opp_opp_England,...,opp_opp_Portugal,opp_opp_Russia,opp_opp_Saudi Arabia,opp_opp_Senegal,opp_opp_Serbia,opp_opp_Spain,opp_opp_Sweden,opp_opp_Switzerland,opp_opp_Tunisia,opp_opp_Uruguay
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
124,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
125,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
126,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
data['Opponent'] = data['Opponent'].apply(lambda x: "opp_" + x)

In [23]:
data

,Team,Opponent,Goal Scored,Ball Possession %,Attempts,On-Target,Off-Target,Blocked,Corners,Offsides,...,Distance Covered (Kms),Fouls Committed,Yellow Card,Yellow & Red,Red,Man of the Match,1st Goal,Round,PSO,Goals in PSO
0,Russia,opp_opp_Saudi Arabia,5,40,13,7,3,3,6,3,...,118,22,0,0,0,1,12.000000,0,0,0
1,Saudi Arabia,opp_opp_Russia,0,60,6,0,3,3,2,1,...,105,10,0,0,0,0,39.457447,0,0,0
2,Egypt,opp_opp_Uruguay,0,43,8,3,3,2,0,1,...,112,12,2,0,0,0,39.457447,0,0,0
3,Uruguay,opp_opp_Egypt,1,57,14,4,6,4,5,1,...,111,6,0,0,0,1,89.000000,0,0,0
4,Morocco,opp_opp_Iran,0,64,13,3,6,4,5,0,...,101,22,1,0,0,0,39.457447,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,England,opp_opp_Croatia,1,46,11,1,6,4,4,3,...,148,14,1,0,0,0,5.000000,3,0,0
124,Belgium,opp_opp_England,2,43,12,4,3,5,4,1,...,108,11,1,0,0,1,4.000000,4,0,0
125,England,opp_opp_Belgium,0,57,15,5,7,3,5,0,...,110,5,2,0,0,0,39.457447,4,0,0
126,France,opp_opp_Croatia,4,39,8,6,1,1,2,1,...,99,14,2,0,0,1,18.000000,5,0,0


In [24]:
data_concat = pd.concat([data, pd.get_dummies(data['Team']), pd.get_dummies(data['Opponent'])], axis=1)

In [25]:
data_concat.drop(['Team', 'Opponent'], axis=1, inplace=True)

In [26]:
data_concat

,Goal Scored,Ball Possession %,Attempts,On-Target,Off-Target,Blocked,Corners,Offsides,Free Kicks,Saves,...,opp_opp_Portugal,opp_opp_Russia,opp_opp_Saudi Arabia,opp_opp_Senegal,opp_opp_Serbia,opp_opp_Spain,opp_opp_Sweden,opp_opp_Switzerland,opp_opp_Tunisia,opp_opp_Uruguay
0,5,40,13,7,3,3,6,3,11,0,...,0,0,1,0,0,0,0,0,0,0
1,0,60,6,0,3,3,2,1,25,2,...,0,1,0,0,0,0,0,0,0,0
2,0,43,8,3,3,2,0,1,7,3,...,0,0,0,0,0,0,0,0,0,1
3,1,57,14,4,6,4,5,1,13,3,...,0,0,0,0,0,0,0,0,0,0
4,0,64,13,3,6,4,5,0,14,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,1,46,11,1,6,4,4,3,24,5,...,0,0,0,0,0,0,0,0,0,0
124,2,43,12,4,3,5,4,1,5,5,...,0,0,0,0,0,0,0,0,0,0
125,0,57,15,5,7,3,5,0,12,2,...,0,0,0,0,0,0,0,0,0,0
126,4,39,8,6,1,1,2,1,14,1,...,0,0,0,0,0,0,0,0,0,0


In [27]:
np.sum(data_concat.dtypes == 'object')

0

In [28]:
y = data_concat['Man of the Match']
X = data_concat.drop('Man of the Match', axis=1)

In [29]:
scaler = RobustScaler()

X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

In [31]:
sk_model = MLPClassifier(hidden_layer_sizes=(16, 16))
sk_model.fit(X_train, y_train)

E:\Users\Rony\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(16, 16))

In [32]:
inputs = tf.keras.Input(shape=(85,))
x = tf.keras.layers.Dense(128, activation=tf.nn.relu)(inputs)
x = tf.keras.layers.Dense(128, activation=tf.nn.relu)(x)
outputs = tf.keras.layers.Dense(2, activation=tf.nn.softmax)(x)

tf_model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [33]:
tf_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [38]:
tf_model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=100
)

Epoch 1/100
3/3 [==============================] - 0s 39ms/step - loss: 0.3490 - accuracy: 0.9437 - val_loss: 0.5880 - val_accuracy: 0.7222
Epoch 2/100
3/3 [==============================] - 0s 13ms/step - loss: 0.3206 - accuracy: 0.9577 - val_loss: 0.5956 - val_accuracy: 0.6667
Epoch 3/100
3/3 [==============================] - 0s 11ms/step - loss: 0.2980 - accuracy: 0.9577 - val_loss: 0.6020 - val_accuracy: 0.6667
Epoch 4/100
3/3 [==============================] - 0s 14ms/step - loss: 0.2752 - accuracy: 0.9718 - val_loss: 0.6064 - val_accuracy: 0.6667
Epoch 5/100
3/3 [==============================] - 0s 13ms/step - loss: 0.2475 - accuracy: 0.9718 - val_loss: 0.6093 - val_accuracy: 0.6667
Epoch 6/100
3/3 [==============================] - 0s 14ms/step - loss: 0.2248 - accuracy: 1.0000 - val_loss: 0.6124 - val_accuracy: 0.6111
Epoch 7/100
3/3 [==============================] - 0s 13ms/step - loss: 0.2125 - accuracy: 0.9718 - val_loss: 0.6187 - val_accuracy: 0.6111
Epoch 8/100
3/3 [===

3/3 [==============================] - 0s 20ms/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 1.3576 - val_accuracy: 0.6111
Epoch 60/100
3/3 [==============================] - 0s 16ms/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 1.3645 - val_accuracy: 0.6111
Epoch 61/100
3/3 [==============================] - 0s 13ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 1.3719 - val_accuracy: 0.6111
Epoch 62/100
3/3 [==============================] - 0s 14ms/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 1.3796 - val_accuracy: 0.6111
Epoch 63/100
3/3 [==============================] - 0s 15ms/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 1.3873 - val_accuracy: 0.6111
Epoch 64/100
3/3 [==============================] - 0s 17ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 1.3953 - val_accuracy: 0.6111
Epoch 65/100
3/3 [==============================] - 0s 17ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 1.4024 - val_accuracy: 0.6111
Epoch 66/100
3/3 [========

In [39]:
sk_score = sk_model.score(X_test, y_test)
tf_score = tf_model.evaluate(X_test, y_test, verbose=False)

In [40]:
print(f"   sklearn Model: {sk_score}")
print(f"TensorFlow Model: {tf_score[1]}")

   sklearn Model: 0.6410256410256411
TensorFlow Model: 0.6666666865348816


In [37]:
X_test.shape

(39, 85)